## <b>Querying a knowledge graph using python</b>

## 1. Set parameters for queries
This Tutorial is based on general action plans of cutting as shown in the presentation. The example scenario is based on the assumption, that a given robot is performing meal preparation tasks in a kitchen. It is given <b>two</b> parameters: <b>a task, and an object</b>. Currently, we are not looking at how these parameters are passed to the robot. Future work will extract these parameters from recipe instructions or natural language text, but right now we just pass them over to the robot. For doing this, we will use a simple dropdown menu in this file.

This step terefore will open a dropdown so you can choose parameters for the following queries. 

In [1]:
import ipywidgets as widgets
from ipywidgets import HBox

# all available parameters
tasks = [('Cutting Action',"cut:CuttingAction"),
        ('Quartering', "cut:Quartering"),
        ('Julienning',"cut:Julienning"),
        ('Halving',"cut:Halving"),
        ('Dicing',"soma:Dicing"),
        ('Cutting',"soma:Cutting"),
        ('Slicing',"soma:Slicing"),
        ('Snipping',"cut:Snipping"),
        ('Slivering',"cut:Slivering"),
        ('Sawing',"cut:Sawing"),
        ('Paring',"cut:Paring"),
        ('Carving',"cut:Carving"),
        ('Mincing',"cut:Mincing"),
        ('Cubing',"cut:Cubing"),
        ('Chopping',"cut:Chopping")]

objects=[('almond', "obo:FOODON_00003523"),
        ('apple', "obo:FOODON_03301710"),
        ('avocado', "obo:FOODON_00003600"),
        ('banana', "obo:FOODON_00004183"),
        ('bean', "obo:FOODON_03301403"),      
        ('cherry', "obo:FOODON_03301240"),
        ('citron', "obo:FOODON_03306596"),
        ('coconut', "obo:FOODON_00003449"),     
        ('cucumber', "obo:FOODON_00003415"),
        ('kiwi', "obo:FOODON_00004387"), 
        ('kumquat', "obo:FOODON_03306597"),
        ('lemon', "obo:FOODON_03301441"),
        ('lime', "obo:FOODON_00003661"),
        ('olive', "obo:FOODON_03317509"),
        ('orange', "obo:FOODON_03309832"),
        ('peach', "obo:FOODON_03315502"), 
        ('pepper', "obo:FOODON_00003520"),
        ('pineapple', "obo:FOODON_00003459"),
        ('pumpkin', "obo:FOODON_00003486"),
        ('strawberry', "obo:FOODON_00003443"),        
        ('squash', "obo:FOODON_00003539"),
        ('tomato', "obo:FOODON_03309927")]

task=""
tobject=""

# Takes a Task of the tasks list. 
# Then makes the previously initialized task variable global and assigns value of the given Task to it 
def chooseTask(Task):
    global task
    task=Task
    
# Takes an Object of the object list.
# Then makes the previously initialized tobject variable global and assigns value of the given Object to it 
def chooseObject(Object):
    global tobject
    tobject=Object
    
# Create the dropdown widgets
task_widget = widgets.Dropdown(options=tasks, description='Task:')
object_widget = widgets.Dropdown(options=objects, description='Object:')

# Define the event handlers
def taskEvent(event):
    chooseTask(event.new)

def objectEvent(event):
    chooseObject(event.new)
    
# Attach the event handlers to the widgets
task_widget.observe(taskEvent, names='value')
object_widget.observe(objectEvent, names='value')

# Combine the widgets using HBox
widgets_display = HBox([task_widget, object_widget])

# Display widgets
widgets_display

# optional: set Parameters manually:
#task = "cut:Quartering"
#tobject = "obo:FOODON_03301710"

## 2. import rdflib for querying the knowledge graph for action parameters
This step is needed to load libraries for execution of the queries

In [2]:
!pip install rdflib
# import rdflib
from rdflib import Graph, Literal, URIRef, Namespace
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc
from rdflib.namespace import OWL, RDF, RDFS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00


--------------------------------------------------------------------------------------------------------------------------------
# <b>Now let us start with querying the knowledge graph!</b>
## Query 1: We can query for the tool to be used for the chosen task:
With this query, a robot can determine which tool shall be used for the cutting task

In [13]:
g = Graph()

# define prefixes to be used in the query 
SOMA = Namespace("http://www.ease-crc.org/ont/SOMA.owl#")
CUT2 = Namespace("http://www.ease-crc.org/ont/situation_awareness#")
CUT = Namespace("http://www.ease-crc.org/ont/food_cutting#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = Namespace("http://purl.obolibrary.org/obo/")
g.bind("owl", OWL)
g.bind("rdfs", RDFS)
g.bind("soma", SOMA)
g.bind("cut2", CUT2)
g.bind("cut", CUT)
g.bind("dul", DUL)
g.bind("obo", OBO)

# the actual query comes here
query = """SELECT ?whattool WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {
     {%s rdfs:subClassOf ?node.
        ?node rdf:type owl:Restriction.}
      UNION
      {%s rdfs:subClassOf ?class.
        ?class rdfs:subClassOf ?node.
        ?node rdf:type owl:Restriction.}
      ?node owl:onProperty soma:hasDisposition.
      ?node owl:someValuesFrom ?collection.
      ?collection owl:intersectionOf ?node2.
      ?node2 rdf:first cut2:Cuttability.
      ?node2 rdf:rest ?toolnode.
      ?toolnode rdf:rest ?collection2.
      ?collection2 rdf:first ?tooluse.
      ?tooluse owl:onProperty soma:affordsTrigger.
      ?tooluse owl:allValuesFrom ?tool.
      ?tool owl:onProperty dul:classifies.
      ?tool owl:allValuesFrom ?whattool.} }""" % (tobject,tobject) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(query):
    print(row.whattool)

http://www.ease-crc.org/ont/SOMA.owl#CuttingTool


# Query 2a: The robot needs to know if additional actions need to be executed:
Additional actions like peeling or core removal can be queried here. This helps the robot to determine if it can proceed with cutting or needs to perform a different task beforehand.

In [6]:
additionalactionquery = """SELECT ?action WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  %s rdfs:subClassOf ?restriction.
  ?restriction owl:onProperty cut:hasPart.
  ?restriction owl:someValuesFrom ?node.
  ?node owl:intersectionOf ?intersec.
  ?intersec rdf:first ?foodpart.
  ?intersec rdf:rest ?node2.
  ?node2 rdf:first ?part.
  ?part owl:onProperty cut:hasEdibility.
  {?part owl:someValuesFrom cut:ShouldBeAvoided.}
  UNION
  {?part owl:someValuesFrom cut:MustBeAvoided.}
  ?foodpart rdfs:subClassOf ?partrestriction.
  ?partrestriction owl:onProperty soma:hasDisposition.
  ?partrestriction owl:someValuesFrom ?partnode.
  ?partnode owl:intersectionOf ?intersection.
  ?intersection rdf:first ?disposition.
  ?intersection rdf:rest ?actionrestriction.
  ?actionrestriction rdf:first ?actionnode.
  ?actionnode owl:onProperty soma:affordsTask.
  ?actionnode owl:someValuesFrom ?action.
} }""" % (tobject) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(additionalactionquery):
    print(row.action)

http://www.ease-crc.org/ont/food_cutting#CoreCutting
http://www.ease-crc.org/ont/food_cutting#CoreScraping
http://www.ease-crc.org/ont/food_cutting#Peeling


# Query 2b: The robot needs to know if prior actions need to be executed:
Prior actions like halving before quartering can be queried here. This helps the robot to determine how often it needs to perform motions.

In [7]:
prioractionquery = """  SELECT ?priortask WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  %s rdfs:subClassOf ?sub.
  ?sub owl:onProperty cut:requiresPriorTask .
  ?sub owl:someValuesFrom ?priortask.
} }""" % (task)

for row in g.query(prioractionquery):
    print(row.priortask)

# Query 3: The robot needs to also know the number of repetitions required for performing the task:
It is important to determine the needed number of repetitions in order to know when a task is finished.

In [8]:
repetitionsquery = """  SELECT ?rep WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  {
     OPTIONAL{ %s rdfs:subClassOf ?action}
        ?action rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {
            ?rep_node owl:hasValue ?val.}
        BIND("= 1" AS ?rep)}
    UNION
    {
       OPTIONAL{ %s rdfs:subClassOf ?action }
        ?action rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {
            ?rep_node owl:minQualifiedCardinality ?val.}
        BIND("more than 1" AS ?rep)}} }""" % (task, task)

for row in g.query(repetitionsquery):
    print(row.rep)

more than 1


# Query 4: The position needed for action execution:
It is important to know where to cut an object.

In [9]:
positionquery = """  SELECT ?position WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  OPTIONAL { %s rdfs:subClassOf ?sub.}
  ?sub rdfs:subClassOf* ?node.
  ?node owl:onProperty cut:affordsPosition.
  ?node owl:someValuesFrom ?position.
} }""" % (task)

for row in g.query(positionquery):
    print(row.position)

http://www.ease-crc.org/ont/food_cutting#slicing_position


# Query 5: What object shall be cut:
When cutting a fruit, for some subtasks the robot does not cut the whole food object but a slice, or a stripe.

In [10]:
objectquery = """  SELECT ?object ?object2 WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  OPTIONAL { %s rdfs:subClassOf ?a.}
  ?a owl:intersectionOf ?obj.
  ?obj rdf:first*/rdf:rest* ?in.
  ?in owl:onProperty cut:hasInputObject .
  ?in owl:someValuesFrom ?object.
  OPTIONAL{
    ?object owl:unionOf ?b.
    ?b rdf:first ?object2.
  }
} }""" % (task)

for row in g.query(objectquery):
    print(row.object, row.object2)

https://krr.triply.cc/.well-known/genid/478e1a077399e08adb4d5f381d17b2a8 http://www.ease-crc.org/ont/food_cutting#Food
